# Opmærksomhedsmekanismer og transformere

En stor ulempe ved rekurrente netværk er, at alle ord i en sekvens har samme indflydelse på resultatet. Dette fører til suboptimal ydeevne med standard LSTM encoder-decoder-modeller til sekvens-til-sekvens-opgaver, såsom navngiven entitetsgenkendelse og maskinoversættelse. I virkeligheden har specifikke ord i inputsekvensen ofte større indflydelse på sekventielle output end andre.

Overvej en sekvens-til-sekvens-model, såsom maskinoversættelse. Den implementeres ved hjælp af to rekurrente netværk, hvor det ene netværk (**encoder**) komprimerer inputsekvensen til en skjult tilstand, og det andet, **decoder**, udfolder denne skjulte tilstand til det oversatte resultat. Problemet med denne tilgang er, at netværkets endelige tilstand har svært ved at huske begyndelsen af en sætning, hvilket resulterer i dårlig modelkvalitet for lange sætninger.

**Opmærksomhedsmekanismer** giver en metode til at vægte den kontekstuelle indflydelse af hver inputvektor på hver outputforudsigelse i RNN. Dette implementeres ved at skabe genveje mellem de mellemliggende tilstande i input-RNN og output-RNN. På denne måde, når vi genererer outputsymbolet $y_t$, tager vi højde for alle skjulte inputtilstande $h_i$ med forskellige vægtkoefficienter $\alpha_{t,i}$. 

![Billede, der viser en encoder/decoder-model med et additivt opmærksomhedslag](../../../../../translated_images/da/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder-modellen med additiv opmærksomhedsmekanisme i [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citeret fra [denne blogpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Opmærksomhedsmatricen $\{\alpha_{i,j}\}$ repræsenterer graden, hvormed visse inputord spiller en rolle i genereringen af et givet ord i outputsekvensen. Nedenfor er et eksempel på en sådan matrix:

![Billede, der viser en prøvejustering fundet af RNNsearch-50, taget fra Bahdanau - arviz.org](../../../../../translated_images/da/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figur taget fra [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Opmærksomhedsmekanismer er ansvarlige for meget af den nuværende eller næsten nuværende state-of-the-art inden for naturlig sprogbehandling. Tilføjelse af opmærksomhed øger dog antallet af modelparametre betydeligt, hvilket førte til skaleringsproblemer med RNN'er. En vigtig begrænsning ved at skalere RNN'er er, at modellernes rekurrente natur gør det udfordrende at batchbehandle og parallelisere træning. I en RNN skal hvert element i en sekvens behandles i rækkefølge, hvilket betyder, at det ikke let kan paralleliseres.

Adoptionen af opmærksomhedsmekanismer kombineret med denne begrænsning førte til skabelsen af de nuværende state-of-the-art Transformer-modeller, som vi kender og bruger i dag, fra BERT til OpenGPT3.

## Transformer-modeller

I stedet for at videregive konteksten af hver tidligere forudsigelse til det næste evalueringsskridt bruger **transformer-modeller** **positionelle kodninger** og opmærksomhed til at fange konteksten af et givet input inden for et angivet tekstvindue. Billedet nedenfor viser, hvordan positionelle kodninger med opmærksomhed kan fange kontekst inden for et givet vindue.

![Animeret GIF, der viser, hvordan evalueringer udføres i transformer-modeller.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Da hver inputposition kortlægges uafhængigt til hver outputposition, kan transformere parallelisere bedre end RNN'er, hvilket muliggør meget større og mere udtryksfulde sprogmodeller. Hver opmærksomhedshoved kan bruges til at lære forskellige relationer mellem ord, hvilket forbedrer opgaver inden for naturlig sprogbehandling.

**BERT** (Bidirectional Encoder Representations from Transformers) er et meget stort flerlagstransformernetværk med 12 lag for *BERT-base* og 24 for *BERT-large*. Modellen fortrænes først på en stor tekstkorpus (Wikipedia + bøger) ved hjælp af usuperviseret træning (forudsige maskerede ord i en sætning). Under fortræningen absorberer modellen et betydeligt niveau af sprogforståelse, som derefter kan udnyttes med andre datasæt ved hjælp af finjustering. Denne proces kaldes **transfer learning**. 

![Billede fra http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/da/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Der findes mange variationer af Transformer-arkitekturer, herunder BERT, DistilBERT, BigBird, OpenGPT3 og flere, som kan finjusteres. [HuggingFace-pakken](https://github.com/huggingface/) giver et bibliotek til træning af mange af disse arkitekturer med PyTorch. 

## Brug af BERT til tekstklassifikation

Lad os se, hvordan vi kan bruge en fortrænet BERT-model til at løse vores traditionelle opgave: sekvensklassifikation. Vi vil klassificere vores oprindelige AG News-datasæt.

Først skal vi indlæse HuggingFace-biblioteket og vores datasæt:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Da vi skal bruge en forudtrænet BERT-model, skal vi anvende en specifik tokenizer. Først vil vi indlæse en tokenizer, der er tilknyttet den forudtrænede BERT-model.

HuggingFace-biblioteket indeholder et arkiv af forudtrænede modeller, som du kan bruge ved blot at angive deres navne som argumenter til `from_pretrained`-funktionerne. Alle nødvendige binære filer til modellen vil automatisk blive downloadet.

Dog vil du nogle gange have behov for at indlæse dine egne modeller. I sådanne tilfælde kan du angive den mappe, der indeholder alle relevante filer, herunder parametre til tokenizer, `config.json`-filen med modelparametre, binære vægte osv.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objektet indeholder `encode`-funktionen, som kan bruges direkte til at kode tekst:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Lad os derefter oprette iteratorer, som vi vil bruge under træningen til at få adgang til dataene. Fordi BERT bruger sin egen kodningsfunktion, skal vi definere en padding-funktion, der ligner `padify`, som vi har defineret tidligere:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

I vores tilfælde vil vi bruge den forudtrænede BERT-model kaldet `bert-base-uncased`. Lad os indlæse modellen ved hjælp af pakken `BertForSequenceClassfication`. Dette sikrer, at vores model allerede har den nødvendige arkitektur til klassifikation, inklusive den endelige klassifikator. Du vil se en advarselsmeddelelse, der angiver, at vægtene for den endelige klassifikator ikke er initialiseret, og modellen vil kræve forudtræning - det er helt i orden, fordi det er præcis, hvad vi er ved at gøre!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nu er vi klar til at begynde træningen! Fordi BERT allerede er forudtrænet, vil vi starte med en ret lille læringsrate for ikke at ødelægge de oprindelige vægte.

Alt det hårde arbejde udføres af `BertForSequenceClassification`-modellen. Når vi kalder modellen på træningsdataene, returnerer den både tab og netværksoutput for input-minibatch. Vi bruger tabet til parameteroptimering (`loss.backward()` udfører den baglæns passering), og `out` til at beregne træningsnøjagtighed ved at sammenligne de opnåede labels `labs` (beregnet ved hjælp af `argmax`) med de forventede `labels`.

For at kontrollere processen akkumulerer vi tab og nøjagtighed over flere iterationer og udskriver dem hver `report_freq` træningscyklus.

Denne træning vil sandsynligvis tage ret lang tid, så vi begrænser antallet af iterationer.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Du kan se (især hvis du øger antallet af iterationer og venter længe nok), at BERT-klassificering giver os ret god nøjagtighed! Det skyldes, at BERT allerede forstår sprogets struktur ganske godt, og vi behøver kun at finjustere den endelige klassifikator. Dog, fordi BERT er en stor model, tager hele træningsprocessen lang tid og kræver betydelige computerressourcer! (GPU, og helst mere end én).

> **Note:** I vores eksempel har vi brugt en af de mindste forudtrænede BERT-modeller. Der findes større modeller, som sandsynligvis vil give bedre resultater.


## Evaluering af modellens ydeevne

Nu kan vi evaluere vores models ydeevne på testdatasættet. Evalueringssløjfen ligner træningssløjfen, men vi må ikke glemme at skifte modellen til evalueringsmodus ved at kalde `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Vigtig pointe

I denne enhed har vi set, hvor nemt det er at tage en forudtrænet sprogmodel fra **transformers**-biblioteket og tilpasse den til vores tekstklassificeringsopgave. På samme måde kan BERT-modeller bruges til enhedsekstraktion, spørgsmål-svar og andre NLP-opgaver.

Transformer-modeller repræsenterer den nuværende state-of-the-art inden for NLP, og i de fleste tilfælde bør det være den første løsning, du eksperimenterer med, når du implementerer skræddersyede NLP-løsninger. Dog er det ekstremt vigtigt at forstå de grundlæggende principper bag rekurrente neurale netværk, som blev diskuteret i dette modul, hvis du ønsker at bygge avancerede neurale modeller.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
